In [ ]:
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
import gc

target_size = (1280,720)
model_input_shape = (299,299)

In [ ]:
def read_and_resize(path,target_size):
    return cv2.resize(cv2.imread(path),target_size)/255

def crop_img(inputImg, scenario,model_input_shape):
    if scenario == 'reward':
        return inputImg[:,335:940]
    elif scenario == 'dialog':
        return inputImg[460:,]
    elif scenario == 'autoIcon':
        icon = inputImg[632:698,351:416] # (66,65,3)
        icon = cv2.resize(icon,model_input_shape)
        return icon

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras import optimizers
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import load_model

def create_resnet_model(n_classes):
    RESmodel=InceptionResNetV2(include_top=True, weights='imagenet')
    RESmodel.layers.pop()
    new_layer = Dense(n_classes, activation='softmax', name='my_dense')
    inp = RESmodel.input
    out = new_layer(RESmodel.layers[-1].output)
    model = Model(inp, out)
    return model

# Train util get a good model

### first model

In [ ]:
img_path_list = glob.glob('./scenario/**/*.*',recursive=True)
print(f"Data Size : {len(img_path_list)}")
img_cls = [os.path.basename(os.path.dirname(i)) for i in img_path_list]
img_cls = [i if i not in ['idle','inQuest'] else 'others' for i in img_cls]
print(f"Data Size : {len(img_cls)}")
classes = list(np.unique(img_cls))
print(classes)
Y = np.zeros((len(img_cls),len(classes)))
for r in range(len(img_cls)):
    Y[r,classes.index(img_cls[r])] = 1
    
X_img = []
for i in img_path_list:
    img = read_and_resize(i,target_size)
    X_img.append(cv2.resize(img,model_input_shape))
X_img = np.array(X_img)
print(X_img.shape)
class_weight = compute_class_weight('balanced',np.unique(img_cls),img_cls)

In [ ]:
accuracy = 0
count = 0
while True:
    clear_output(wait=True)
    count+=1
    print(f"Training count : {count}")   
    model = create_resnet_model(len(classes))
    model.compile(optimizer=Adam(lr=0.005), loss='categorical_crossentropy',metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=1)
    model.fit(X_img, Y,batch_size=6,epochs=200,callbacks=[reduce_lr, early_stopping],class_weight=class_weight)
    score, accuracy = model.evaluate(X_img, Y,batch_size=6)
    print(f"Accuracy : {round(accuracy,3)}")
    if accuracy >= 0.99:
        model.save("AutoQuest_first_model.h5")
        del model, accuracy
        gc.collect()
        break
    else:
        del model, accuracy
        gc.collect()

### second model

In [ ]:
img_path_list = glob.glob('./scenario/**/*.*',recursive=True)
np.random.shuffle(img_path_list)
img_cls = [os.path.basename(os.path.dirname(i)) for i in img_path_list]
img_path_list = [i for j,i in enumerate(img_path_list) if img_cls[j] in ['idle','inQuest']]
img_cls = [i for i in img_cls if i in ['idle','inQuest']]
print(f"Data Size : {len(img_cls)}")
classes = list(np.unique(img_cls))
print(classes)
Y = np.zeros((len(img_cls),len(classes)))
for r in range(len(img_cls)):
    Y[r,classes.index(img_cls[r])] = 1
    
X_img = []
for i in img_path_list:
    img = read_and_resize(i,target_size)
    X_img.append(crop_img(img,'autoIcon',model_input_shape))
X_img = np.array(X_img)
print(X_img.shape)

In [ ]:
accuracy = 0
count = 0
while True:
    clear_output(wait=True)
    count+=1
    print(f"Training count : {count}")
    model = create_resnet_model(len(classes))
    model.compile(optimizer=Adam(lr=0.005), loss='categorical_crossentropy',metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=1)
    model.fit(X_img, Y,batch_size=6,epochs=200,callbacks=[reduce_lr, early_stopping],class_weight=class_weight)
    score, accuracy = model.evaluate(X_img, Y,batch_size=6)
    print(f"Accuracy : {round(accuracy,3)}")
    if accuracy >= 0.99:
        model.save("AutoQuest_second_model.h5")
        del model, accuracy
        gc.collect()
        break
    else:
        del model, accuracy
        gc.collect()

# Model evaluation

In [ ]:
first_classes = ['dialog', 'loading', 'others', 'reward']
second_classes = ['idle', 'inQuest']
def check_scenario(inputImg):
    inputImg_1 = cv2.resize(inputImg,model_input_shape)
    inputImg_1 = np.expand_dims(inputImg_1,0)
    pred = model1.predict(inputImg_1)
    first_scenario = first_classes[np.argmax(pred)]
    if first_scenario != 'others':
        return first_scenario
    else:
        inputImg_2 = crop_img(inputImg,'autoIcon',model_input_shape)
        inputImg_2 = np.expand_dims(inputImg_2,0)
        pred = model2.predict(inputImg_2)
        second_scenario = second_classes[np.argmax(pred)]
        return second_scenario

In [ ]:
model1 = load_model("AutoQuest_first_model.h5")
model2 = load_model("AutoQuest_second_model.h5")

In [ ]:
img_path_list = glob.glob('./scenario/**/*.*',recursive=True)
print(f"Data Size : {len(img_path_list)}")
img_cls = [os.path.basename(os.path.dirname(i)) for i in img_path_list]
    
X_img = []
for i in img_path_list:
    img = read_and_resize(i,target_size)
    X_img.append(img)
X_img = np.array(X_img)
output_scenario = [check_scenario(i) for i in X_img]
print(confusion_matrix(img_cls,output_scenario))